

<h1 style="text-align:center;">Exploring Survival on the Titanic</h1>



**Team members** <br>
Ronald  Santhosh-ENG19CS0263​ <br>
Sabeeha  Tabassum – ENG19CS0270​ <br>
Sakina Bathool – ENG19CS0278​<br>
Shushant  Rishav -  ENG19CS0306​<br>

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install sklearn
# %pip install plotly

In [2]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Sklearn
from sklearn.metrics import roc_curve, auc ,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score


# Loading Data

In [3]:
train_df = pd.read_csv('./Statics/Titanic/train.csv')
test_df = pd.read_csv('./Statics/Titanic/test.csv')
print(train_df.shape)
train_df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<h4>  Features Description : </h4>
<p>
Survival : Survival(0/1) <br>
PassengerId : Unique Id of a passenger. <br>
Pclass : Ticket class <br>
Sex : Male/Female   <br>
Age : Age in years<br>     
Sibsp : num of siblings / spouses aboard the Titanic <br>     
Parch : num of parents / children aboard the Titanic     <br>
Ticket : Ticket number     <br>
Fare : Passenger fare     <br>
Cabin : Cabin number     <br>
Embarked : Port of Embarkation<br>
</p>

# Exploratory Data Analysis 1 (Data Overview)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
def update_feature_lists():
    binary = [c for c in train_df.columns if train_df[c].nunique() == 2]
    numerical = ['Age',
                        'Pclass',
                        'Fare',
                        'PassengerId',
                        'Parch']
    categorical = [c for c in train_df.columns if (c not in numerical) & (c not in binary)]
    return binary, numerical,categorical
    
binary_columns,true_numerical_columns, categorical_columns = update_feature_lists()

In [6]:
total = train_df.shape[0]
missing_df = []
cardinality_df = []
for col in train_df.columns:
    missing_df.append([col, train_df[col].count(), total])
    cardinality = train_df[col].nunique()
    if cardinality > 2 and col != 'MachineIdentifier':
        cardinality_df.append([col, cardinality])
    
missing_df = pd.DataFrame(missing_df, columns=['Column', 'Number of records', 'Total']).sort_values("Number of records", ascending=False)
cardinality_df = pd.DataFrame(cardinality_df, columns=['Column', 'Cardinality']).sort_values("Cardinality", ascending=False)
type_df = [['Binary columns', len(binary_columns)], ['Numerical columns', len(true_numerical_columns)], ['Categorical columns', len(categorical_columns)]]
type_df = pd.DataFrame(type_df, columns=['Type', 'Column count']).sort_values('Column count', ascending=True)

In [7]:
def missing_values_table(df):
        mis_val = df.isnull().sum()                                     # Total missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)             # Percentage of missing values
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)   # Make a table with the results
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})      # Rename the columns
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)                  # Sort the table by percentage of missing descending
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")                     # Print some summary information
        return mis_val_table_ren_columns                                # Return the dataframe with missing information

In [8]:
print(missing_values_table(train_df))

Your selected dataframe has 12 columns.
There are 3 columns that have missing values.
          Missing Values  % of Total Values
Cabin                687               77.1
Age                  177               19.9
Embarked               2                0.2


In [9]:
fig = px.bar(missing_df,x=['Total','Number of records'],height=500,width=1000,y='Column',title="Missing Data")
fig.update_layout(barmode='group',paper_bgcolor="aliceblue")
fig.show()

In [10]:
fig = px.bar(cardinality_df,x='Cardinality',y='Column',height=500,width=1000,title='Cardinality Count')
fig.update_layout(paper_bgcolor="aliceblue")
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

# Exploratory Data Analysis 2 (Feature Overview)

In [11]:
train_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [12]:
fig = px.histogram(train_df,x='Sex',color='Survived',width=1000)
fig.update_layout(yaxis_title="Number Of Passengers",barmode='group',bargroupgap=0,paper_bgcolor="aliceblue")
fig.show()

In [13]:
fig=px.histogram(train_df,color="Survived",x="Age",width=1000,marginal="box",hover_data=train_df.columns)
fig.update_layout(yaxis_title="Number Of Passengers",paper_bgcolor="aliceblue")
fig.show()


In [14]:
fig=px.histogram(train_df,color="Survived",x="Fare",width=1000)
fig.update_layout(barmode='group',bargroupgap=0,paper_bgcolor="aliceblue", yaxis_title="Number Of Passengers" ,title="Fare Vs. survivor")
fig.show()


In [15]:
fig = px.histogram(train_df, x="Age",color='Survived',facet_row='Sex',facet_col="Survived",width=1000)
fig.update_layout(paper_bgcolor="aliceblue",title={'text' : "Gender Vs Age Survival rate",'x':0.5,'xanchor': 'center'})
fig.show()

In [16]:
fig = px.histogram(train_df,x='Embarked',color='Survived',width=1000)
fig.update_layout(barmode='group',bargroupgap=0,paper_bgcolor="aliceblue",yaxis_title="Number Of Passengers",title= "Embarked Vs survivor")
fig.show()


In [17]:
fig = px.histogram(train_df,x='Pclass',color='Survived',width=1000)
fig.update_layout(yaxis_title="Number Of Passengers",barmode='group',bargroupgap=0,paper_bgcolor="aliceblue",title="Passenger Class Vs survivor")
fig.show()

In [18]:
fig = px.histogram(train_df, x="Embarked",color='Survived',facet_col='Pclass',facet_row='Sex',category_orders={'Pclass':[1,2,3],'Embarked':['C','Q','S']},width=1000)
fig.update_layout(barmode='group',bargroupgap=0,paper_bgcolor="aliceblue",title={'text' : "Embarked, Pclass and Gender Vs Survivor",'x':0.5,'xanchor': 'center'})
fig.show()

In [19]:
fig = px.histogram(train_df,x="Embarked",color='Survived',facet_col="Pclass",category_orders={"Pclass": [1,2,3]},width=1000)
fig.update_layout(yaxis_title="Number Of Passengers",barmode='group',bargroupgap=0,paper_bgcolor="aliceblue",title = "Embarked and Passenger Class Vs survivor")
fig.show()

In [20]:
fig = px.histogram(train_df, x="Age",color='Survived',facet_row='Pclass',facet_col="Survived",category_orders={"Pclass": [1,2,3]},width=1000)
fig.update_layout(paper_bgcolor="aliceblue",title={'text' : "Pclass Vs Age Survival rate",'x':0.5,'xanchor': 'center'})
fig.show()

<p>SibSp and  Parch combined shows the total number of relatives, a person has onboard</p>

In [21]:
data = [train_df,test_df]
for dataset in data:
    dataset['Relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['Relatives'] > 0, 'Alone'] = 0
    dataset.loc[dataset['Relatives'] == 0, 'Alone'] = 1
    dataset['Alone'] = dataset['Alone'].astype(int)
train_df['Alone'].value_counts()

1    537
0    354
Name: Alone, dtype: int64

In [22]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Relatives,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0,1


In [23]:
fig = px.histogram(train_df,x="Relatives",color="Survived",width=1000)
fig.update_layout(yaxis_title="Number Of Passengers",barmode='group',bargroupgap=0,paper_bgcolor="aliceblue",title="Relatives Vs survivor")
fig.show()

# Data Preprocessing

<p>Dropping Passenger ID
As This Feature Is Not Useful</p>

In [24]:
train_df = train_df.drop(['PassengerId'], axis=1)

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   891 non-null    int64  
 1   Pclass     891 non-null    int64  
 2   Name       891 non-null    object 
 3   Sex        891 non-null    object 
 4   Age        714 non-null    float64
 5   SibSp      891 non-null    int64  
 6   Parch      891 non-null    int64  
 7   Ticket     891 non-null    object 
 8   Fare       891 non-null    float64
 9   Cabin      204 non-null    object 
 10  Embarked   889 non-null    object 
 11  Relatives  891 non-null    int64  
 12  Alone      891 non-null    int32  
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 87.1+ KB


<p> **** Catagorical Columns : Name, Sex, Ticket,Cabin and Embarked <br></p>

<p>Since Fare is in <b>FLOAT</b> We Have to Change it to <b>INT</b></p>

In [26]:
train_df['Fare'] = train_df['Fare'].astype(int)
test_df['Fare'] = test_df['Fare'].fillna(0)
test_df['Fare'] = test_df['Fare'].astype(int)
# train_df.info()

In [27]:
def drop_ele(df):
    df.drop(['Name', 'Ticket','Cabin'], axis=1, inplace=True)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df.dropna(inplace= True)
    return  df
train_df = drop_ele(train_df)
test_df = drop_ele(test_df)

<p>Since Gender is catagorical column We have to convert it to Int</p>

In [28]:
genders = {"male": 0, "female": 1}
train_df['Sex'] = train_df['Sex'].map(genders)
test_df['Sex'] = test_df['Sex'].map(genders)
# train_df.info()

<p>Convert ‘Embarked' feature into numeric</p>

In [29]:
ports = {"S": 0, "C": 1, "Q": 2}
train_df['Embarked'] = train_df['Embarked'].map(ports)
test_df['Embarked'] = test_df['Embarked'].map(ports)
# train_df.info()

#### Creating Age Groups

In [30]:
data = [train_df,test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

train_df['Age'].value_counts()

4    303
6    147
3    106
5    102
2     92
1     71
0     68
Name: Age, dtype: int64

In [31]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Survived   889 non-null    int64
 1   Pclass     889 non-null    int64
 2   Sex        889 non-null    int64
 3   Age        889 non-null    int32
 4   SibSp      889 non-null    int64
 5   Parch      889 non-null    int64
 6   Fare       889 non-null    int32
 7   Embarked   889 non-null    int64
 8   Relatives  889 non-null    int64
 9   Alone      889 non-null    int32
dtypes: int32(3), int64(7)
memory usage: 66.0 KB


In [32]:
train_df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,Alone
0,0,3,0,2,1,0,7,0,1,0
1,1,1,1,5,1,0,71,1,1,0
2,1,3,1,3,0,0,7,0,0,1
3,1,1,1,5,1,0,53,0,1,0
4,0,3,0,5,0,0,8,0,0,1


## Building ML Models

In [33]:
labels = train_df.pop('Survived')

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(train_df, labels, test_size=0.25)

### Random Forest:

In [35]:
random_forest = RandomForestClassifier()

random_forest.fit(X_train, Y_train)
Y_prediction = random_forest.predict(X_test)

acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

### Decision Tree

In [36]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)  
Y_pred = decision_tree.predict(X_test)  
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

### Stochastic Gradient Descent (SGD) Classifier

In [37]:
sgd = SGDClassifier(max_iter=100, tol=None)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
sgd.score(X_train, Y_train)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

### Gaussian Naive Bayes:

In [38]:
gaussian = GaussianNB() 
gaussian.fit(X_train, Y_train)  
Y_pred = gaussian.predict(X_test)  
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)


### KNN Classifier

In [39]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)  
Y_pred = knn.predict(X_test)  
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

### Perceptron:

In [40]:
perceptron = Perceptron(max_iter=100)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)


In [41]:
results = pd.DataFrame({
    'Model': ['KNN', 
              'Random Forest', 'Gaussian Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [ acc_knn, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head()

,Model
Score,
93.39,Random Forest
93.39,Decision Tree
83.93,KNN
79.58,Gaussian Naive Bayes
79.58,Stochastic Gradient Decent


<p>*** Since Random forest score is the max we will proceed with that </p>

### Setting up Hyperparameter for RandomForest

In [42]:
model = RandomForestClassifier(bootstrap=True, 
                        criterion='gini',
                        max_features='sqrt',
                        min_samples_leaf=1,
                        min_samples_split=10, 
                        n_estimators=150, 
                        n_jobs=-1, 
                        oob_score=True, 
                        random_state=1,
                        warm_start= False,
                        verbose=0)
model.fit(X_train,Y_train)
acc_model = model.score(X_train, Y_train)
print("Score:", round(acc_model*100,2), "%")
print("oob score:", round(model.oob_score_, 4)*100, "%")

Score: 89.49 %
oob score: 81.98 %


#### K-Fold Cross Validation:

In [43]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train, Y_train, cv=cv, scoring = "accuracy",n_jobs=-1)
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.79104478 0.89552239 0.82089552 0.86567164 0.73134328 0.7761194
 0.78787879 0.83333333 0.87878788 0.77272727 0.7761194  0.85074627
 0.82089552 0.89552239 0.82089552 0.80597015 0.78787879 0.84848485
 0.81818182 0.78787879 0.8358209  0.82089552 0.85074627 0.85074627
 0.74626866 0.7761194  0.83333333 0.86363636 0.77272727 0.84848485]
Mean: 0.8188225538971807
Standard Deviation: 0.04145506946983256


In [44]:

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, Y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc
model = RandomForestClassifier(bootstrap=True, 
                        criterion='gini',
                        max_features='sqrt',
                        min_samples_leaf=1,
                        min_samples_split=10, 
                        n_estimators=150, 
                        n_jobs=-1, 
                        oob_score=True, 
                        # random_state=1, #Since it smooths the curve 
                        verbose=0, 
                        warm_start=False)
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100]
train_results = []
test_results = []
for estimator in n_estimators:
   model.fit(X_train, Y_train)

   train_pred = model.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)

   test_pred = model.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, test_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
print("roc_auc =",roc_auc)

roc_auc = 0.8109177215189873


In [45]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(
    go.Scatter(x=n_estimators, y=train_results),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=n_estimators, y=test_results),
    row=2, col=1
)
fig.update_layout(height=600, width=800, title_text="Auc Score Vs n_estimators")
fig.show()


## Feature Importance

In [46]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(15)

,importance
feature,
Sex,0.409
Fare,0.198
Pclass,0.118
Age,0.100
Relatives,0.059
Embarked,0.048
SibSp,0.031
Parch,0.026
Alone,0.011


In [47]:
fig = px.bar(importances,orientation='h')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

### Confusion Matrix

In [48]:
predictions = cross_val_predict(model, X_train, Y_train, cv=3)
matrix = confusion_matrix(Y_train, predictions)
auc_scr = accuracy_score(Y_test,Y_pred)

In [49]:
print(classification_report(Y_train, predictions))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       405
           1       0.81      0.73      0.77       261

    accuracy                           0.83       666
   macro avg       0.82      0.81      0.81       666
weighted avg       0.82      0.83      0.82       666



In [50]:
fig = ff.create_annotated_heatmap(matrix,colorscale='blues',showscale=True)
fig.update_layout(paper_bgcolor="aliceblue",width=500,title="Confusion Matrix")
fig.show()

### Making Predictions on Test Data

In [51]:
result = test_df[['PassengerId']].copy()
test_df = test_df.drop(['PassengerId'], axis=1)
test_df.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,Alone
0,3,0,5,0,0,7,2,0,1
1,3,1,6,1,0,7,0,1,0
2,2,0,6,0,0,9,2,0,1
3,3,0,3,0,0,8,0,0,1
4,3,1,2,1,1,12,0,2,0


In [52]:
pred = model.predict(test_df)
result['Pred'] = pred 
result.head(10)
result.to_csv("./Statics/Titanic/test_result.csv")